In [89]:
import os
import pandas as pd
import numpy as np
import dateparser
import spacy
import re
import fr_core_news_sm
from spacy.lang.fr.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf 

In [90]:
p = r'C:\Users\david\Downloads'
f = os.path.join(p, '', 'essai1249.json')
# df = pd.read_csv(f, lines=True, parse_dates=['date'], date_parser=dateparser.parse)
df = pd.read_json(f, lines=True)

### 1ere solution pour changer le format de date

In [91]:
print(df.shape)
df.head(5)

(376, 9)


,nom_auteur,prenom_auteur,titre,tags,note,date,appreciations,sexe_redacteur,commentaire
0,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",5.0,20 février 2020,5,Femme,Voici un livre extraordinaire sur l'absurdité ...
1,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,22 mars 2021,6,Femme,Kumaresan et Saroja sont amoureux fous. Malheu...
2,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,23 mai 2020,7,Femme,"Pour subvenir à ses besoins, Kumaresan quitte ..."
3,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,01 février 2020,9,Femme,"Malheureusement, les castes sont toujours d'ac..."
4,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,28 janvier 2020,17,Femme,"En ouvrant le bûcher, je suis totalement sorti..."


In [92]:
df['commentaire'][0]

'Voici un livre extraordinaire sur l\'absurdité des gens et des croyances. Comme ma lecture précédente, cette histoire a un petit goût de "Roméo et Juliette" : deux jeunes gens qui n\'auraient pas dû se rencontrer et encore moins s\'aimer, d\'après le système de castes très rigide indien. Mais l\'amour ne se contrôle pas. \nJ\'ai été profondément touchée par l\'histoire de nos deux jeunes personnages principaux, prêts à tout pour leurs sentiments, même à faire face aux courroux de la mère et des villageois aux mentalités plus qu\'arriérées : comment peut-on se comporter de la sorte, avoir des envies de meurtre contre deux jeunes amoureux, tout simplement parce qu\'ils ne font pas partie de la même caste ? \nLa jeune femme a beaucoup de courage de ne pas craquer face aux remarques acerbes et aux langues aussi acérées que des lames de couteaux ! C\'est terrible de voir ce que Saroja a sacrifié par amour : son confort, sa vie aisée, son existence très protégée.\nA travers cette histoire, 

In [93]:
# si pas assez de valeurs dans la colonnes df = df[df.columns[df.isna().sum() == 0]]
df = df.dropna().reset_index().drop(['index'], axis=1)
df.shape

(376, 9)

In [94]:
df.isna().sum()

nom_auteur        0
prenom_auteur     0
titre             0
tags              0
note              0
date              0
appreciations     0
sexe_redacteur    0
commentaire       0
dtype: int64

In [95]:
dict_mapping = {'janvier' : 'January',
                'février' : 'February',
                'fevrier' : 'February',
                'mars' : 'March',
                'avril' : 'April',
                'mai' : 'May',
                'juin' : 'June',
                'juillet': 'July',
                'août' : 'August',
                'septembre' : 'September',
                'octobre' : 'October',
                'novembre' : 'November',
                'décembre' : 'December'}  

In [96]:
def format_date(df, col, dict_):
    date_list = df[col].tolist()
    new_date_list, index_problem = [], []
    for i, el in enumerate(date_list):
        for fr_word, en_word in dict_.items():
            if fr_word in el:
                el = el.strip().replace(fr_word, en_word)
        new_date_list.append(el)
    df['new_date'] = pd.Series(new_date_list)
    return df

In [97]:
df = format_date(df, 'date', dict_mapping)

In [98]:
df['new_date']

0      20 February 2020
1         22 March 2021
2           23 May 2020
3      01 February 2020
4       28 January 2020
             ...       
371    02 February 2019
372    07 February 2019
373    23 February 2019
374    26 February 2019
375       03 March 2019
Name: new_date, Length: 376, dtype: object

In [99]:
df.head()

,nom_auteur,prenom_auteur,titre,tags,note,date,appreciations,sexe_redacteur,commentaire,new_date
0,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",5.0,20 février 2020,5,Femme,Voici un livre extraordinaire sur l'absurdité ...,20 February 2020
1,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,22 mars 2021,6,Femme,Kumaresan et Saroja sont amoureux fous. Malheu...,22 March 2021
2,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,23 mai 2020,7,Femme,"Pour subvenir à ses besoins, Kumaresan quitte ...",23 May 2020
3,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,01 février 2020,9,Femme,"Malheureusement, les castes sont toujours d'ac...",01 February 2020
4,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,28 janvier 2020,17,Femme,"En ouvrant le bûcher, je suis totalement sorti...",28 January 2020


In [100]:
def preprocessing(df):
    df["titre"] = df["titre"].apply(lambda x: x.strip())
    df["note"] = df["note"].apply(lambda x:np.nan if x=='' else float(x))
    df["Date"] = pd.to_datetime(df["new_date"]).dt.strftime('%Y-%m-%d')
    df["Date"] = pd.to_datetime(df["Date"])
    df["month"] =  df["Date"].dt.month 
    df["day_name"] = df["Date"].dt.day_name()
    df["day"] = df["Date"].dt.day
    df["homme"] = df["sexe_redacteur"].apply(lambda x: 1 if x=="Homme" else 0)
    df["femme"] = df["sexe_redacteur"].apply(lambda x: 1 if x=="Femme" else 0)
    return df

In [101]:
df = preprocessing(df)
df.head()

,nom_auteur,prenom_auteur,titre,tags,note,date,appreciations,sexe_redacteur,commentaire,new_date,Date,month,day_name,day,homme,femme
0,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",5.0,20 février 2020,5,Femme,Voici un livre extraordinaire sur l'absurdité ...,20 February 2020,2020-02-20,2,Thursday,20,0,1
1,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,22 mars 2021,6,Femme,Kumaresan et Saroja sont amoureux fous. Malheu...,22 March 2021,2021-03-22,3,Monday,22,0,1
2,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,23 mai 2020,7,Femme,"Pour subvenir à ses besoins, Kumaresan quitte ...",23 May 2020,2020-05-23,5,Saturday,23,0,1
3,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,01 février 2020,9,Femme,"Malheureusement, les castes sont toujours d'ac...",01 February 2020,2020-02-01,2,Saturday,1,0,1
4,Perumal,Murugan,Le bûcher,"roman,roman d'amour,sentiments,traditions,litt...",4.0,28 janvier 2020,17,Femme,"En ouvrant le bûcher, je suis totalement sorti...",28 January 2020,2020-01-28,1,Tuesday,28,0,1


### Text preprocessing

In [102]:
def text_preprocessing1(df, col):
    import string
    from spacy.lang.fr.stop_words import STOP_WORDS
    nlp = fr_core_news_sm.load()
    stopwords = list(STOP_WORDS)
    stopwords += ["neuf", "qu", "quelqu"]
    corpus = df["commentaire"].tolist()
    comms = []
    for el in corpus:
        out = el.replace("'", ' ')
        out = out.translate(str.maketrans(' ',' ',string.punctuation))
        out = out.replace(':', '').replace('\n', ' ').strip()
        out = [token.lemma_ for token in nlp(out) if (token.lemma_ not in stopwords) and (len(token) > 1) and (token.text not in stopwords) or (token.text=='x' or token.text=='y' or token.text=='i')]
        out = ' '.join(out)
        comms.append(out)
    df["commentaire_clean"] = pd.Series(comms)

    return df

In [103]:
df = text_preprocessing1(df, "commentaire")
df["commentaire_clean"][0]

'livre extraordinaire absurdité croyance lecture précédent histoire petit goût roméo Juliette jeune devoir rencontrer aimer système caste rigide indien amour contrôle profondément toucher histoire jeune personnage principal prêt sentiment faire face courroux mère villageois mentalité arriéré peuton comporter sorte envie meurtre contre jeune amoureux simplement partie caste    jeune femme beaucoup courage craquer face remarque acerbe langue acérer lame couteau terrible voir Saroja sacrifier amour confort vie aisé existence protéger travers histoire auteur découvrir mentalité indien coutume intransigeant campagne brosse portrait vivant donne voir ville emplir saveur grouillante vie écriture fluide agréable ravir découvrir auteur connaître auparavant donner envi lire roman'

### Alternative Text preprocessing

In [67]:
def text_preprocessing2(df, col):
    from spacy.lang.fr.stop_words import STOP_WORDS
    stopwords = list(STOP_WORDS)
    nlp = fr_core_news_sm.load()
    stopwords += ["neuf", "qu", "quelqu"]
    corpus = df["commentaire"].tolist()
    comms = []
    for el in corpus:
        s_chars = r"""-"/(),!.:+=[{()}]\^$?#*'%"""
        trans = str.maketrans(s_chars, ' '*len(s_chars))
        el = el.translate(trans)
        el = [token.lemma_ for token in nlp(el) if (token.lemma_ not in stopwords) and (len(token) > 1) and (token.text not in stopwords) or (token.text=='x' or token.text=='y' or token.text=='i')]
        el = ' '.join(el).replace('\n', ' ').replace('  ', ' ').replace('   ',' ')
        comms.append(el)
    df["commentaire_clean"] = pd.Series(comms)

    return df

In [68]:
df = text_preprocessing2(df, "commentaire")
df["commentaire_clean"][0]

'livre extraordinaire absurdité croyance lecture précédent histoire petit goût roméo Juliette jeune devoir rencontrer aimer système caste rigide indien amour contrôle  profondément toucher histoire jeune personnage principal prêt sentiment faire face courroux mère villageois mentalité arriéré  comporter sorte envie meurtre contre jeune amoureux simplement partie caste jeune femme beaucoup courage craquer face remarque acerbe langue acérer lame couteau  terrible voir Saroja sacrifier amour  confort vie aisé existence protéger travers histoire auteur découvrir mentalité indien coutume intransigeant campagne brosse portrait vivant donne voir ville emplir saveur grouillante vie écriture fluide agréable ravir découvrir auteur connaître auparavant donner envi lire roman'

### Ngrams

In [104]:
def ngrams_top(df, col, ngram_range=(1,1), n=None):
    from sklearn.feature_extraction.text import CountVectorizer
    from spacy.lang.fr.stop_words import STOP_WORDS
    corpus = df[col]
    stop_words = list(STOP_WORDS) + ["neuf", "qu", "quelqu"]                              # Stop-words
    vec = CountVectorizer(stop_words = stop_words, ngram_range=ngram_range).fit(corpus)   # CountVectorizer
    bag_of_words = vec.transform(corpus)                                                  # Sparse Matrix shape(376, 49060)
    sum_words = bag_of_words.sum(axis=0)                                                  # Vector        shape(  1, 49060)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]     # Counter-list   [('livre', 312),..
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)                    # Sorted Count [('houellebecq', 646),
    total_list=words_freq[:n]                                                             # Select n top rows
    df=pd.DataFrame(total_list,columns=["text","count"])                                  # Splitting in 'text', 'count' cols
    return df

In [105]:
ngrams_top_df = ngrams_top(df, "commentaire_clean", ngram_range=(1,2), n=15)
ngrams_top_df

,text,count
0,houellebecq,633
1,roman,428
2,bien,358
3,livre,350
4,auteur,336
5,vie,301
6,amour,245
7,sérotonine,242
8,monde,221
9,personnage,208


In [106]:
ngrams_top2_df = ngrams_top(df, "commentaire_clean", ngram_range=(2,2), n=15)
ngrams_top2_df

,text,count
0,michel houellebecq,154
1,ingénieur agronom,45
2,florentclaude labrouste,40
3,personnage principal,37
4,gilet jaune,34
5,carte territoire,30
6,ministère agriculture,29
7,bien sûr,24
8,roman houellebecq,21
9,46 an,21


### Vectorization

In [107]:
def vectorization(df, col):
    df2 = df.copy()
    mask = df["commentaire_clean"].apply(lambda x: type(x)==str)
    df = df.loc[mask,:]
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="out_of_vocab") 
    tokenizer.fit_on_texts(df["commentaire_clean"])
    df["commentaire_clean_encoded"] = tokenizer.texts_to_sequences(df["commentaire_clean"])
    reviews_pad = tf.keras.preprocessing.sequence.pad_sequences(df["commentaire_clean_encoded"], padding="post")
    return df, reviews_pad

In [109]:
df, reviews_pad = vectorization(df, "commentaire_clean")
df['commentaire_clean_encoded']

0      [5, 1, 1, 1, 32, 233, 21, 28, 387, 1, 1, 56, 1...
1      [108, 148, 120, 1, 638, 188, 108, 51, 335, 1, ...
2      [1, 244, 108, 388, 169, 1, 1, 1, 348, 144, 226...
3      [638, 188, 510, 226, 593, 905, 1, 169, 810, 1,...
4      [512, 513, 299, 160, 910, 803, 288, 189, 226, ...
                             ...                        
371    [145, 1, 1, 374, 1, 241, 200, 1, 1, 1, 1, 98, ...
372                             [1, 116, 35, 96, 526, 1]
373    [10, 69, 22, 24, 1, 1, 46, 836, 9, 1, 675, 7, ...
374    [1, 264, 1, 86, 1, 1, 145, 34, 208, 385, 212, ...
375                                                [152]
Name: commentaire_clean_encoded, Length: 376, dtype: object

In [110]:
reviews_pad

array([[  5,   1,   1, ...,   0,   0,   0],
       [108, 148, 120, ...,   0,   0,   0],
       [  1, 244, 108, ...,   0,   0,   0],
       ...,
       [ 10,  69,  22, ...,   0,   0,   0],
       [  1, 264,   1, ...,   0,   0,   0],
       [152,   0,   0, ...,   0,   0,   0]])

### TF-IDF

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000
stop_words = list(STOP_WORDS) + ["neuf", "qu", "quelqu"] 

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=stop_words)
tfidf = tfidf_vectorizer.fit_transform(df['commentaire_clean'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

In [112]:
print(tfidf.shape)
tfidf.todense()

(376, 1000)


matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.07987833, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [113]:
print(tfidf_feature_names.shape)
tfidf_feature_names

(1000,)


array(['2019', '46', 'abandonner', 'aborde', 'aborder', 'absence',
       'absolument', 'absurde', 'accepter', 'accompagner', 'accord',
       'acerbe', 'acheter', 'acteur', 'action', 'actualité', 'actuel',
       'adore', 'adorer', 'adulte', 'affectif', 'agacer', 'agir',
       'agricole', 'agriculteur', 'agriculture', 'agro', 'agronom',
       'agréable', 'aide', 'aider', 'ailleurs', 'aime', 'aimer', 'air',
       'alcool', 'alcoolique', 'allemand', 'aller', 'amateur', 'ambiance',
       'amener', 'ami', 'amie', 'amitié', 'amour', 'amoureux', 'an',
       'analyse', 'ancien', 'animal', 'annoncer', 'année', 'anti',
       'anticiper', 'antidépresseur', 'antihéros', 'apparaître',
       'appartement', 'appartenir', 'appel', 'appeler', 'apporte',
       'apporter', 'apprendre', 'apprécier', 'appuyer', 'argent',
       'aristocrate', 'arrive', 'arriver', 'arrière', 'arrêter',
       'assumer', 'atil', 'atteindre', 'attendre', 'attention', 'aucun',
       'audelà', 'aujourd', 'auprès', 'a

### Latent Dirichlet Analysis

In [114]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words=stop_words)
tf = tf_vectorizer.fit_transform(df['commentaire_clean'])
tf_feature_names = tf_vectorizer.get_feature_names_out()

### Non-Negative Matrix Factorization (NMF)

In [115]:
from sklearn.decomposition import LatentDirichletAllocation, NMF

no_topics = 20

nmf = NMF(n_components=no_topics, random_state=1, alpha_W=.1, l1_ratio=.5, init='nndsvd').fit(tfidf) # alpha_H=.1

lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

### Display & Evaluer les topics

In [116]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [117]:
no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 1:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 2:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 3:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 4:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 5:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 6:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 7:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 8:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 9:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 10:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 11:
île expliquer faux fantasme fan famille falloir faire facilement facile
Topic 12:
île expliquer fa

### Splitting dfs

In [119]:
n = len(df) // 7

In [120]:
for i in range(7):
    globals() [f"sample{i+1}"] = df.iloc[n*i:n*(i+1), :]
    globals() [f"sample{i+1}"].reset_index().drop(['index'],axis=1)

In [123]:
sample1.shape, sample2.shape, sample3.shape, sample4.shape, sample5.shape, sample6.shape, sample7.shape

((53, 18), (53, 18), (53, 18), (53, 18), (53, 18), (53, 18), (53, 18))

In [201]:
sample2

,nom_auteur,prenom_auteur,titre,note,date,appreciations,sexe_redacteur,commentaire,new_date,Date,month,day_name,day,homme,femme
100,Carlos,Ruiz Zafón,L'ombre du vent,4,08 août 2018,27,Femme,"[Ce roman ne tarit pas d'éloges. du coup, je l...",08 August 2018,2018-08-08,8,Wednesday,8,0,1
101,Carlos,Ruiz Zafón,L'ombre du vent,5,18 juillet 2019,27,Femme,"[Chronique de l'Espagne sous Franco, histoire ...",18 July 2019,2019-07-18,7,Thursday,18,0,1
102,Carlos,Ruiz Zafón,L'ombre du vent,5,01 mai 2021,27,Non renseigné,"[Wouah, alors, ça, ça envoie ! Pour utiliser u...",01 May 2021,2021-05-01,5,Saturday,1,0,0
103,Carlos,Ruiz Zafón,L'ombre du vent,4,16 juin 2021,27,Femme,"[J'ai découvert, par hasard, en me baladant su...",16 June 2021,2021-06-16,6,Wednesday,16,0,1
104,Carlos,Ruiz Zafón,L'ombre du vent,5,12 octobre 2021,27,Homme,[Comment commencer cette chronique autrement q...,12 October 2021,2021-10-12,10,Tuesday,12,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Carlos,Ruiz Zafón,L'ombre du vent,4,21 mars 2012,20,Non renseigné,"[Avant de commencer, je ne connaissais pas, (o...",21 March 2012,2012-03-21,3,Wednesday,21,0,0
196,Carlos,Ruiz Zafón,L'ombre du vent,5,01 mai 2012,20,Femme,[Cela faisait longtemps qu'un livre ne m'avait...,01 May 2012,2012-05-01,5,Tuesday,1,0,1
197,Carlos,Ruiz Zafón,L'ombre du vent,5,25 mai 2012,20,Non renseigné,[Un étrange sentiment m'envahit au moment où j...,25 May 2012,2012-05-25,5,Friday,25,0,0
198,Carlos,Ruiz Zafón,L'ombre du vent,4,15 février 2021,19,Femme,"[amours impossibles, des frissons, des mystère...",15 February 2021,2021-02-15,2,Monday,15,0,1
